In [11]:
import pandas as pd
import numpy as np
import os
import json

In [12]:
model_path="/home/sumbhati/ingenious/LM-pretraining/models/regular_bert_13_06_2022_18:22:00/step_375000"
data=pd.DataFrame(columns=["cb", "copa", "wsc", "boolq", "multirc", "wic"], index=[f"superglue_run_{i}" for i in range(1, 21)])
task2metric={"boolq":"acc", "cb": "avg_f1", "copa":"acc", "multirc":"f1", "record":"f1", "wic": "acc", "wsc": "acc"}
for run in range(1, 21):
    if run>5:
        tasks=["cb", "copa", "wsc"]
    else:
        tasks=["cb", "copa", "wsc", "boolq", "multirc", "wic"]
    l=[]
    for task in tasks:
        with open(f"{model_path}/superglue_run_{run}/runs/{task}/val_metrics.json") as f:
            d=json.load(f)
            score=d[task]["metrics"]["minor"][task2metric[task]]
            l.append(round(100.0*float(score), 2))
    if run>5:
        l.extend([np.nan]*3)
    data.loc[f"superglue_run_{run}"]=l
data.loc["average"]=data.mean(axis=0, skipna=True)
data.loc["stddev"]=data.std(axis=0, skipna=True)
data.loc["decrement_avg"]=[np.nan for i in range(6)]
data.loc["maximum"]=data.max(axis=0, skipna=True)
data.loc["decrement_max"]=[np.nan for i in range(6)]
data.to_csv(f"{model_path}/superglue_results.csv")

In [13]:
data

,cb,copa,wsc,boolq,multirc,wic
superglue_run_1,76.55,60.0,58.65,73.88,61.6,68.34
superglue_run_2,77.83,61.0,61.54,73.12,0.0,68.65
superglue_run_3,65.09,62.0,65.38,69.72,61.98,66.77
superglue_run_4,72.28,61.0,63.46,73.94,59.53,68.18
superglue_run_5,72.09,58.0,55.77,73.85,62.46,68.34
superglue_run_6,72.17,66.0,61.54,NaN,NaN,NaN
superglue_run_7,73.51,65.0,62.5,NaN,NaN,NaN
superglue_run_8,84.19,59.0,63.46,NaN,NaN,NaN
superglue_run_9,76.18,61.0,59.62,NaN,NaN,NaN
superglue_run_10,67.74,73.0,57.69,NaN,NaN,NaN
